In [11]:
import os

In [12]:
%pwd

'f:\\Practice\\EndtoEnd\\ML-Project'

In [13]:
os.chdir("../")

In [14]:
%pwd

'f:\\Practice\\EndtoEnd'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [16]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

In [30]:
from box.exceptions import BoxValueError
import yaml
from src.mlproject.logging import logger
import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = self.read_yaml(config_filepath)
        self.params = self.read_yaml(params_filepath)
        self.schema = self.read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    @ensure_annotations
    def read_yaml(self,path_to_yaml: Path) -> ConfigBox:
        print("HHHHHHHHHHH")
        try:
            with open(path_to_yaml) as yaml_file:
                content = yaml.safe_load(yaml_file)
                logger.info(f"yaml file: {path_to_yaml} loaded successfully")
                return ConfigBox(content)
        except BoxValueError:
            raise ValueError("yaml file is empty")
        except Exception as e:
            raise e
    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config


In [26]:
import pandas as pd
import os
from src.mlproject.logging import logger
from sklearn.linear_model import ElasticNet
import joblib

In [31]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))




In [32]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'